In [1]:
# imports 
from path import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
#import statsapi
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
PFL = pd.read_csv("C:/Users/GeorgeSmith/Documents/Syracuse/IST 718/Final Project/PFL_Rolling_DB2.csv")

In [3]:
PFL2 = PFL.dropna()

In [4]:
PFL2.head()

,Date,Fighter,Sig. Str. Leg,Sig. Str. Leg Attempts,Opp Sig. Str. Leg,Opp Sig. Str. Leg Attempts,Sig. Str. Ground Landed,Sig. Str. Ground Attempts,Opp Sig. Str. Ground Landed,Opp Sig. Str. Ground Attempts,...,Opp Outer Zone,Sig. Str. Per KnockDown,Opp Sig. Str. Per KnockDown,Sig. Str. Punches Diff,Sig. Str. Diff,Sig. Str. Leg Diff,Sig. Str. Ground Diff,Odds,Total Wins,Win
5,6/7/2018 0:00,Steven Siler,0,1,1,2,0,1,13,13,...,0.0,0.000000,31.000000,0.000000,-0.275287,-0.500000,-1.000000,1100,1,1
8,6/7/2018 0:00,Marcos Galvao,9,15,3,6,6,6,4,7,...,0.0,0.000000,20.666667,-0.333333,0.045777,0.100000,0.428571,110,0,0
9,6/7/2018 0:00,Nazareno Malegarie,3,6,9,15,4,7,6,6,...,0.0,20.666667,0.000000,0.333333,-0.045777,-0.100000,-0.428571,-140,1,1
11,6/7/2018 0:00,Valdrin Istrefi,3,5,4,6,2,7,71,92,...,0.0,0.000000,0.000000,0.000000,-0.232020,-0.066667,-0.486025,310,0,0
12,6/7/2018 0:00,Alex Nicholson,6,6,11,20,0,0,0,0,...,0.0,0.000000,24.000000,0.000000,-0.062058,0.450000,0.000000,-135,1,1


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras import losses
import seaborn
from scipy.stats import poisson,skellam

In [6]:
# Create x data set to train set and split 
X = PFL2.copy()
X.drop(["Total Wins"], axis=1, inplace=True)
X.drop(["Win"], axis=1, inplace=True)
X.drop(["Date"], axis=1, inplace=True)
X.drop(["Fighter"], axis=1, inplace=True)
X.drop(["Height"], axis=1, inplace=True)
X.head()

,Sig. Str. Leg,Sig. Str. Leg Attempts,Opp Sig. Str. Leg,Opp Sig. Str. Leg Attempts,Sig. Str. Ground Landed,Sig. Str. Ground Attempts,Opp Sig. Str. Ground Landed,Opp Sig. Str. Ground Attempts,Total Strikes Landed,Total Strikes Attempts,...,Opp Inner Zone,Outer Zone,Opp Outer Zone,Sig. Str. Per KnockDown,Opp Sig. Str. Per KnockDown,Sig. Str. Punches Diff,Sig. Str. Diff,Sig. Str. Leg Diff,Sig. Str. Ground Diff,Odds
5,0,1,1,2,0,1,13,13,7,29,...,0.0,0.0,0.0,0.000000,31.000000,0.000000,-0.275287,-0.500000,-1.000000,1100
8,9,15,3,6,6,6,4,7,61,140,...,0.0,0.0,0.0,0.000000,20.666667,-0.333333,0.045777,0.100000,0.428571,110
9,3,6,9,15,4,7,6,6,62,159,...,0.0,0.0,0.0,20.666667,0.000000,0.333333,-0.045777,-0.100000,-0.428571,-140
11,3,5,4,6,2,7,71,92,11,23,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.232020,-0.066667,-0.486025,310
12,6,6,11,20,0,0,0,0,17,67,...,0.0,0.0,0.0,0.000000,24.000000,0.000000,-0.062058,0.450000,0.000000,-135


In [7]:
# Create y data set to train set and split 
y = PFL2["Win"]

In [8]:
# train test and split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Random Forest Model 
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [10]:
# Making predictions using the testing data Random Forest 
predictions = rf_model.predict(X_test_scaled)

In [11]:
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [12]:
# Display Predictions 
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,40,17
Actual 1,12,41


Accuracy Score : 0.7363636363636363
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        57
           1       0.71      0.77      0.74        53

    accuracy                           0.74       110
   macro avg       0.74      0.74      0.74       110
weighted avg       0.74      0.74      0.74       110



In [13]:
importances = rf_model.feature_importances_

sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1217231870634441, 'Odds'),
 (0.05530970100193494, 'Sig. Str. Diff'),
 (0.053025162185451925, 'Points'),
 (0.03372440101695282, 'Opp Total Strikes %'),
 (0.033152612822406426, 'Opp Total Strikes Landed'),
 (0.032613222979547475, 'Total Strikes %'),
 (0.03231284801299634, 'Total Strikes Landed'),
 (0.025131599718774305, 'Sig. Str. Leg Diff'),
 (0.024803420064386253, 'Total Strikes Attempts'),
 (0.024411239907425338, 'Opp Sig. Str. Ground %'),
 (0.023231030812152217, 'Opp Sig. Str. Ground Landed'),
 (0.022701231188050452, 'Sig. Str. Punches Diff'),
 (0.02207610979893205, 'Opp Total Strikes Attempts'),
 (0.0198168924431875, 'Sig. Str. Ground Attempts'),
 (0.01956956855200881, 'Opp Sig. Str. Leg %'),
 (0.019434154161106415, 'Top Strike Speed'),
 (0.019048909887510915, 'Opp Standing Time'),
 (0.01817803381514004, 'Opp Takedown %'),
 (0.0181126537718363, 'Standing Time'),
 (0.01771311161832972, 'Sig. Str. Ground %'),
 (0.017257629841343915, 'Sig. Str. Leg %'),
 (0.016872402005682918, 'Sig

In [14]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample( X_train, y_train)
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

Counter(y_resampled)
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
confusion_matrix(y_test, y_pred)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.58      0.58      0.55      0.58      0.56      0.32        57
          1       0.55      0.55      0.58      0.55      0.56      0.32        53

avg / total       0.56      0.56      0.56      0.56      0.56      0.32       110



C:\Users\GeorgeSmith\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\GeorgeSmith\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
import pydotplus
from IPython.display import Image

In [16]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()
    

Learning rate:  0.05
Accuracy score (training): 0.851
Accuracy score (validation): 0.727

Learning rate:  0.1
Accuracy score (training): 0.866
Accuracy score (validation): 0.745

Learning rate:  0.25
Accuracy score (training): 0.918
Accuracy score (validation): 0.655

Learning rate:  0.5
Accuracy score (training): 0.970
Accuracy score (validation): 0.682

Learning rate:  0.75
Accuracy score (training): 0.979
Accuracy score (validation): 0.745

Learning rate:  1
Accuracy score (training): 0.982
Accuracy score (validation): 0.645



In [17]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.25,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(20)

,Prediction,Actual
0,0,1
1,1,0
2,0,0
3,0,1
4,1,1
5,1,1
6,0,0
7,0,1
8,1,1
9,0,1


In [18]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.6545454545454545


In [19]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,35,22
Actual 1,16,37


In [20]:
# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.69      0.61      0.65        57
           1       0.63      0.70      0.66        53

    accuracy                           0.65       110
   macro avg       0.66      0.66      0.65       110
weighted avg       0.66      0.65      0.65       110



In [21]:
# Graph tree
dot_data = tree.export_graphviz(
    classifier.estimators_[9, 0],
    out_file=None, filled=True,
    rounded=True,
    special_characters=True,
    proportion=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [22]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)


confusion_matrix(y_test, y_pred)

C:\Users\GeorgeSmith\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[37, 20],
       [25, 28]], dtype=int64)

In [23]:
balanced_accuracy_score(y_test, y_pred)

0.5887123469049984

In [24]:
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
importances = model.feature_importances_

importances_data = sorted(zip(model.feature_importances_, X.columns), reverse=True)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
df_features = pd.DataFrame(importances_data)

In [ ]:
df_top_ten_features = df_features[:10]
df_top_ten_features.columns =['percentage', 'feature name']

In [ ]:
import hvplot.pandas

In [ ]:
df_top_ten_features.hvplot.bar( x='feature name', y='percentage', rot=60)

In [25]:
# tensor flow imports 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras import losses
import seaborn
from scipy.stats import poisson,skellam

In [27]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=52))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32)

Epoch 1/100
11/11 [==============================] - 0s 2ms/step - loss: 0.7345 - accuracy: 0.5653
Epoch 2/100
11/11 [==============================] - 0s 898us/step - loss: 0.6630 - accuracy: 0.5957
Epoch 3/100
11/11 [==============================] - 0s 798us/step - loss: 0.6278 - accuracy: 0.6109
Epoch 4/100
11/11 [==============================] - 0s 798us/step - loss: 0.6010 - accuracy: 0.6596
Epoch 5/100
11/11 [==============================] - 0s 898us/step - loss: 0.5807 - accuracy: 0.6687
Epoch 6/100
11/11 [==============================] - 0s 798us/step - loss: 0.5630 - accuracy: 0.6960
Epoch 7/100
11/11 [==============================] - 0s 797us/step - loss: 0.5474 - accuracy: 0.7143
Epoch 8/100
11/11 [==============================] - 0s 698us/step - loss: 0.5347 - accuracy: 0.7356
Epoch 9/100
11/11 [==============================] - 0s 797us/step - loss: 0.5245 - accuracy: 0.7325
Epoch 10/100
11/11 [==============================] - 0s 798us/step - loss: 0.5145 - accuracy

In [28]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.7059 - accuracy: 0.6909 - 98ms/epoch - 25ms/step
Normal Neural Network - Loss: 0.7059475779533386, Accuracy: 0.6909090876579285


In [29]:
predicted = model.predict(X_test_scaled)

In [30]:
results = pd.DataFrame({
    "Actual": y_test,
    "Predicted": predicted.ravel()
})

In [31]:
results.head(10)

,Actual,Predicted
410,1,0.626233
150,0,0.164962
523,0,0.476352
398,1,0.885862
497,1,0.720771
440,1,0.956091
291,0,0.876179
159,1,0.463670
641,1,0.787107
606,1,0.660098


In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                1696      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [52]:
importances = model.feature_importances_

sorted(zip(model.feature_importances_, X.columns), reverse=True)

AttributeError: 'Sequential' object has no attribute 'feature_importances_'

In [46]:
number_hidden_nodes = 8
number_input_features = 92

nn = Sequential()
# Hidden layer
nn.add(
    Dense(units=number_hidden_nodes, input_dim=number_input_features, activation="relu")
)
# Output layer
nn.add(Dense(units=1, activation="linear"))

TypeError: 'History' object is not subscriptable

In [ ]:
number_input_features = 84
hidden_nodes_layer1 = 252
hidden_nodes_layer2 = 126

nn = Sequential()
# First hidden layer
nn.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(Dense(units=1, activation="linear"))

In [ ]:
# Compile model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model_2 = nn.fit(X, y, validation_split=0.3, epochs=20)

In [ ]:
# Train vs test for shallow net
plt.plot(model_1.history["loss"])
plt.plot(model_1.history["val_loss"])
plt.title("loss_function - Training Vs. Validation - 1 hidden layer")
plt.legend(["train", "test"])
plt.show()

In [ ]:
plt.plot(model_2.history["loss"])
plt.plot(model_2.history["val_loss"])
plt.title("loss_function - Training Vs. Validation - 2 hidden layers")
plt.legend(["train", "test"])
plt.show()

In [ ]:
model_loss, model_accuracy = model_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")